**Import Libraries**

In [24]:
%run challenge-1.ipynb

import pandas as pd
import warnings
from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VideoMachine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

, encoding='latin-1', header=None

In [27]:
dataset = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')
dataset.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
data = dataset.sample(1000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [28]:
data['text_processed'] = data['text'].apply(clean_up)
data['text_processed'] = data['text_processed'].apply(tokenize)
data['text_processed'] = data['text_processed'].apply(stem_and_lemmatize)
data['text_processed'] = data['text_processed'].apply(remove_stopwords)

In [29]:
data.head(3)

,target,id,date,flag,user,text,text_processed
566098,0,2206726787,Wed Jun 17 06:58:16 PDT 2009,NO_QUERY,hannah106,ipod is being gay. will not download the games,"[ipod, gay, download, game]"
206968,0,1973365176,Sat May 30 11:00:30 PDT 2009,NO_QUERY,Adam__Carter,shaheen = same song,"[shaheen, song]"
1219085,4,1989901070,Mon Jun 01 03:12:20 PDT 2009,NO_QUERY,roskelld,@pinkzibra Looking forward to that again. I've...,"[pinkzibra, forward, started, show, season, we..."


In [30]:
data['text_processed']

566098                           [ipod, gay, download, game]
206968                                       [shaheen, song]
1219085    [pinkzibra, forward, started, show, season, we...
297611                          [theglamlife, twitter, long]
1583624      [listening, rock, age, work, weee, reelpersona]
                                 ...                        
394263                                [wish, beatles, purse]
355930     [fux, friday, uggh, hot, jam, amp, spot, b, li...
701606          [lalala, miss, school, friend, daily, basis]
946503     [beinggayle, dod, watched, notebook, kso, dami...
1232072    [serialseb, quot, long, piece, string, quot, p...
Name: text_processed, Length: 1000, dtype: object

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [31]:
from nltk.probability import FreqDist
import numpy as np
n_words = 5000

f_dist = FreqDist([item for sublist in data.text_processed for item in sublist])
top_words = [i for i in f_dist][:n_words]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [37]:
def find_features(text):
    words = set(text)
    features = {}
    for w in top_words:
        features[w] = (w in words)
    return features

# Transforming target column into boolean Values
# data['target'] = data['target'].map({4: True, 0: False})
feature_set = [(find_features(text),target) for text,target in list(zip(data.text_processed, data.target))]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [41]:
train = feature_set[:round(len(data)*0.75)]
test = feature_set[round(len(data)*0.25):]

In [42]:
classifier = nltk.NaiveBayesClassifier.train(train)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [43]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, test))*100)
classifier.show_most_informative_features(15)

Classifier accuracy percent: 86.4
Most Informative Features
                   sorry = True            False : True   =      8.4 : 1.0
                    cute = True             True : False  =      6.1 : 1.0
                    luck = True             True : False  =      5.4 : 1.0
                   watch = True             True : False  =      5.4 : 1.0
                       r = True             True : False  =      4.6 : 1.0
                    love = True             True : False  =      4.2 : 1.0
                   great = True             True : False  =      4.1 : 1.0
                   photo = True             True : False  =      3.9 : 1.0
                     run = True             True : False  =      3.9 : 1.0
                     yay = True             True : False  =      3.9 : 1.0
                    wish = True            False : True   =      3.9 : 1.0
                   bored = True            False : True   =      3.4 : 1.0
                homework = True         

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [44]:
lst = classifier.most_informative_features(500)

In [45]:
top_cols = [i[0] for i in lst]

In [46]:
def find_features_2(text):
    words = set(text)
    features = {}
    for w in top_cols:
        features[w] = (w in words)
    return features

feature_set_2 = [(find_features_2(text),target) for text,target in list(zip(data.text_processed, data.target))]

In [47]:
train_t = feature_set_2[:round(len(data)*0.75)]
test_t = feature_set_2[round(len(data)*0.25):]

In [48]:
classifier_2 = nltk.NaiveBayesClassifier.train(train_t)

In [49]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier_2, test_t))*100)

Classifier accuracy percent: 73.2


**Notes**
***
As we can see above, we can achieve acceptable performance with 10x less features

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [3]:
import re
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords

class WordClassifier:
    
    def __init__(self, path, n_words=5000, sample=50000, top_words='',model='', data= '',original=''):
        
        self.path = path
        self.n_words = n_words
        self.top_words = top_words
        self.model = model
        self.data = data
        self.sample = sample
        self.dataset = original
        
    def prepare_data(self):
        self.dataset = pd.read_csv(self.path,encoding='ISO-8859-1')
        self.dataset.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
        data = self.dataset.sample(self.sample)
        
        print('Inicializing class, please wait.')
        data['text_processed'] = data['text'].apply(self.clean_up)
        print('Tokenize.')
        data['text_processed'] = data['text_processed'].apply(self.tokenize)
        print('Stem/Lem.')
        data['text_processed'] = data['text_processed'].apply(self.stem_and_lemmatize)
        print('Stopwords.')
        data['text_processed'] = data['text_processed'].apply(self.remove_stopwords)          

        f_dist = FreqDist([item for sublist in data.text_processed for item in sublist])
        self.top_words = [i for i in f_dist][:self.n_words+1]

        lst = data.text_processed.apply(self.find_features)
        df = pd.DataFrame(list(lst), columns=self.top_words)
        df['target'] = list(data['target'])
        self.data = df
        print('Class created')
    

    def clean_up(self, s):
        x = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,3}\b', ' ',s)
        x = re.findall(r' ?[a-zA-Z]+', x)
        return " ".join(x).lower()
    

    def tokenize(self,s):
        return word_tokenize(s)
    

    def stem_and_lemmatize(self,l):
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(x) for x in l]
    

    def remove_stopwords(self,l):
#         return list(set(l).difference(stopwords.words()))
        return [word for word in l if not word in stopwords.words()]

    def find_features(self, text):
        words = set(text)
        features = []
        for w in self.top_words:
            if w in words:
                features.append(1)
            else:
                features.append(0)
        return features
    
    def create_model(self, d1= 300, d2=100, batch=20, activation = 'sigmoid'):
        import numpy as np
        from keras.models import Sequential
        from keras.layers import Dense
        import tensorflow as tf
        from keras.callbacks import ReduceLROnPlateau, EarlyStopping
        
        tf.config.set_visible_devices([], 'GPU')
        
        X = np.array(self.data.drop(columns='target')).astype(np.float32)
        y = np.array(self.data.target.map({4: True, 0: False})).astype(np.float32)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)  
        display(X_train.shape)
        display(y_train.shape)
        print('Creating model, please wait.')
        
        print(len(self.data.columns)-1)
        # Build the model.
        model = Sequential([ 
            Dense(d1, activation='relu', input_shape=(len(self.data.columns)-1,)), 
            Dense(d1, activation='relu'),
            Dense(d2, activation='relu'),
            Dense(1, activation=activation)])

        # Compile the model.
        model.compile(
          optimizer='adam', 
          loss=tf.keras.losses.binary_crossentropy, 
          metrics=['accuracy'],
        )
        
        # setting up callbacks
        callbacks = [ ReduceLROnPlateau(monitor='loss', patience=10, cooldown=0),
              EarlyStopping(monitor='accuracy', min_delta=1e-4, patience=10)]
            
        # Train the model.
        history = model.fit(
            X_train,
            y_train, 
            epochs=50,
            batch_size=batch, 
            verbose = 1,
#             use_multiprocessing = 1,
            callbacks = callbacks,
        )
        
        print('Evaluate model\n')
        self.model = history
        # Evaluate the model.
        model.evaluate(
          X_test,
          y_test
        )
        
        return self.model
    
    def get_data(self):
        return self.data
    
    def data_feed(self, df):
        self.data = df

In [4]:
new_model = WordClassifier('training.1600000.processed.noemoticon.csv')
new_model.prepare_data()

Inicializing class, please wait.
Tokenize.
Stem/Lem.
Stopwords.
Class created


In [52]:
new_model.create_model(300,100,50)

(40000, 5000)

(40000,)

Creating model, please wait.
5000
Epoch 1/50
800/800 [==============================] - 7s 8ms/step - loss: 0.5644 - accuracy: 0.7046 - lr: 0.0010
Epoch 2/50
800/800 [==============================] - 7s 8ms/step - loss: 0.4508 - accuracy: 0.7872 - lr: 0.0010
Epoch 3/50
800/800 [==============================] - 6s 8ms/step - loss: 0.2896 - accuracy: 0.8684 - lr: 0.0010
Epoch 4/50
800/800 [==============================] - 6s 8ms/step - loss: 0.1422 - accuracy: 0.9357 - lr: 0.0010
Epoch 5/50
800/800 [==============================] - 6s 8ms/step - loss: 0.0940 - accuracy: 0.9569 - lr: 0.0010
Epoch 6/50
800/800 [==============================] - 6s 8ms/step - loss: 0.0796 - accuracy: 0.9619 - lr: 0.0010
Epoch 7/50
800/800 [==============================] - 6s 8ms/step - loss: 0.0709 - accuracy: 0.9646 - lr: 0.0010
Epoch 8/50
800/800 [==============================] - 6s 8ms/step - loss: 0.0699 - accuracy: 0.9649 - lr: 0.0010
Epoch 9/50
800/800 [==============================] - 6s 8ms/s